In [ ]:
import datetime
from datetime import date

import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from data_preparing import sessions_download, read_db_paths, df_tasks_prepare, emotions_habits_df_prepare, \
    tasks_read_metrics, tasks_motivation
from data_preparing import number_tasks_per_day,number_tasks_per_day_per_task, number_hard_tasks_per_m
from data_preparing import check_goals_difficulty, amount_tasks_done_life_areas, tasks_done_per_month
from data_preparing import amount_complited_tasks_permonth, amount_complited_tasks_per_list_month, \
    check_error_rate_predicting_ttc,amount_new_tasks_per_day
from data_preparing import after_metric_ttc

from data_preparing import *
from bot_answers_analysis import load_emotions_habits_values
from recommendation_ML import *

from data_preparing import memory_usage

# import


import warnings
warnings.filterwarnings('ignore')

In [ ]:
DB_ADASH1, DB_TODO, DB_ACTWATCH, DB_GADGETBRIDGE, DB_ADASH2, DB_ADASH3, DB_ADASH4, DB_EMOTIONS_TEST = read_db_paths()


df_sessions = pd.DataFrame(sessions_download())
df_sessions.rename(columns={'description': 'subject'}, inplace=True)


df_sessions=df_sessions[df_sessions.duration>0]
# df_sessions


df_tasks, df_joined = df_tasks_prepare(DB_TODO)
# df_tasks
# df_joined


df_tasks_2 = tasks_read_metrics(df_tasks, df_joined)
# df_tasks_2


df_tasks_3 = tasks_motivation(df_tasks_2)
# df_tasks_3


df_emotions, df_habits, df_emotions1 = emotions_habits_df_prepare()


df_tasks_3_t = after_features_assumpted(df=df_sessions, df1=df_tasks_3, df_tasks_3=df_tasks_3,
                                        df_emotions1=df_emotions1)


# memory_usage()


In [ ]:
df_tasks_3_t.head()

In [ ]:
df_emotions1


In [ ]:
df_sessions

In [ ]:
def floor_dt(dt, interval=10):
    """rounding datetime column to 10 min intervals"""

    replace = (dt.minute // interval) * interval
    return dt.replace(minute=replace, second=0, microsecond=0)

# DB_EMOTIONS_TEST = read_db_paths()[-1]
emotions_habits = pd.read_csv(DB_EMOTIONS_TEST)
emotions_values, habits_values, emotions_and_habits_values = load_emotions_habits_values()
# emotions_habits

emotions_habits['datetime'] = emotions_habits.date.astype('str') + ' ' + emotions_habits.time.astype('str')
emotions_habits['datetime'] = emotions_habits['datetime'].astype('datetime64')

emotions_habits.drop(['date', 'time'], axis=1, inplace=True)
emotions_habits['datetime_rnd'] = emotions_habits.datetime.apply(floor_dt)
emotions_habits['emo_hab'] = np.where(emotions_habits['act'].isin(emotions_values.keys()), 0, 1)

emotions_habits = emotions_habits.groupby('datetime_rnd')['act'].agg(list).reset_index()  # .iloc[1,:]


# GENERATION get dummies from list of emotions (act column) grouped by 10min periods
emotions_habits = emotions_habits.join(emotions_habits['act'].str.join('|').str.get_dummies())

emotions_habits.datetime_rnd = emotions_habits.datetime_rnd.dt.strftime('%Y-%m-%d %H:%M:%S')#.set_index('datetime_rnd')
emotions_habits = emotions_habits.drop('act', axis=1)
emotions_habits.set_index('datetime_rnd', inplace=True)
# print(emotions_habits)


def merge_close_timeslots(df):
    """MERGING THE MOST CLOSE REGISTERED ROWS-TIMESLOTS FOR EMOTIONS HABITS DF"""
    list_of_dd = df.index.to_list()
    # print(list_of_dd)

    list_of_id_todrop = []
    indd=0

    for kk in list_of_dd:
        if indd+2<=len(list_of_dd):
            deltaa=abs(pd.to_datetime(kk) - pd.to_datetime(list_of_dd[indd+1]))
            if deltaa < pd.Timedelta('1 hours'):
                # print(df.loc[kk])
                # print('---')
                df.loc[kk] = df.loc[kk] + df.loc[list_of_dd[indd+1]]
                list_of_id_todrop.append(list_of_dd[indd+1])

                # print(df.loc[list_of_dd[indd+1]])
                # print('---')
                #
                # print(df.loc[kk])
                # print(indd, kk, list_of_dd[indd+1]) # for checking
            indd+=1
    for ki in list_of_id_todrop:
        df.drop(ki, inplace=True)
    return df



emotions_habits = merge_close_timeslots(emotions_habits)


# MAKING DF FROM SESSIONS
def group_by_period(dff):
        dff['for_count']=dff['subject'].map(type) == str
        #number of sessions per day
        number_sessions_per_day = dff.groupby(pd.Grouper(key='start', axis=0,
                                             freq='H')).agg({'for_count':sum}).reset_index()

        #amount of hours tracked per day
        amount_of_hours_tracked_per_day = dff.groupby([pd.Grouper(key='start', axis=0,
                                             freq='H'),]).agg({'duration':sum})/ 3600
        # number_sessions_per_day.start = number_sessions_per_day.start.astype(str)

        out=pd.concat([number_sessions_per_day.set_index('start'),amount_of_hours_tracked_per_day], axis=1).reset_index()
        return out

df_sessions_for_good = group_by_period(df_sessions)
df_sessions_for_good = df_sessions_for_good.rename(columns={'for_count':'amount_sessions', 'start':'datetime_rnd'})
df_sessions_for_good = df_sessions_for_good.drop(columns={'duration'})

# FROM HERE WE ARE TAKING NUMBER OF SESSIONS IN SPECIFIC TIMESLOTS
df_sessions_for_good.datetime_rnd = df_sessions_for_good.datetime_rnd.dt.strftime('%Y-%m-%d %H:%M:%S')#.set_index('datetime_rnd')

df_sessions_for_good.set_index('datetime_rnd', inplace=True)



# DETECTING NOT REGISTERED TIME OF WHEN TIME WAS DONE

df_sessions1 = df_sessions.groupby(['subject']).agg({'stop': max}).reset_index()

df_tasks_3_t_ = df_tasks_3_t.groupby(['completed_datetime']).agg({'subject':"value_counts"})
# df_tasks_3_t_

tasks_done_per_days =  df_tasks_3_t_.groupby(['completed_datetime']).agg({'subject':sum}).reset_index()
# tasks_done_per_days

# ADDING HOURS MINUTES TO TASK_DONE DF BASED ON TIMESTAMPS IN EMOTIONS DF
import random
def add_random_hours_min(k):
    # detecting timeslots in emotions habits df to JOIN on them later
    list_of_timeslots = emotions_habits.reset_index().datetime_rnd.astype('datetime64').dt.strftime("%H:%M:%S").unique()
    return k +' '+ np.random.choice(list_of_timeslots)

tasks_done_per_days.completed_datetime = tasks_done_per_days.completed_datetime.astype(str).apply(lambda x:
                                                                                                add_random_hours_min(x))
tasks_done_per_days.completed_datetime = tasks_done_per_days.completed_datetime.astype('datetime64')
tasks_done_per_days = tasks_done_per_days.rename(columns={'completed_datetime':'datetime_rnd','subject':'tasks_done'})
tasks_done_per_days = tasks_done_per_days.set_index('datetime_rnd')


def find_nearest_date(df1, df_emo):
    """
    finding nearest date in df_emo and write it to df1

    #TODO too many times reseting the index. but now pipeline used in inner functions

    :param df1: df of sessions with generated hours minutes
    :param df_emo: emotions habits dataframe
    :return: df with changed time
    """

    df1 = df1.reset_index()
    df_emo = df_emo.reset_index()
    df1.datetime_rnd = df1.datetime_rnd.astype('datetime64')
    df_emo.datetime_rnd = df_emo.datetime_rnd.astype('datetime64')

    df2 = df1.copy() #CONTroVERSIVE

    # return df1.datetime_rnd
    for i in df2.datetime_rnd:
        # print(i)
        minidx_ = abs(i - df_emo['datetime_rnd']).argmin()
        delta = abs(i - df_emo.loc[minidx_,'datetime_rnd'])
        # print(i, df_emo.loc[[minidx_]]['datetime_rnd'] , delta, delta/pd.Timedelta('1 hour'))

        if delta/pd.Timedelta('1 hour') < 5:
            index = df2.index[df2.datetime_rnd==i]
            # print(df1.loc[index, 'datetime_rnd'])
            # print('BEFORE',df_emo.loc[minidx_,'datetime_rnd'])
            df2.loc[index, 'datetime_rnd'] = df_emo.loc[minidx_,'datetime_rnd']
            # print('CHANGE', df1.loc[index, 'datetime_rnd'])

        # print()
        # print(df_emo.loc[[minidx_]])
    return df2



df_sessions_for_good_ = find_nearest_date(df_sessions_for_good, emotions_habits)
tasks_done_per_days_ = find_nearest_date(tasks_done_per_days, emotions_habits)

#summing up duplicates
df_sessions_for_good_ = df_sessions_for_good_.reset_index().groupby('datetime_rnd').agg({'amount_sessions': sum})

#just test
# df_sessions_for_good_[df_sessions_for_good_.datetime_rnd=='2022-08-09 10:50:00']
# df_sessions_for_good_[df_sessions_for_good_.amount_sessions>0]


# df_sessions_for_good_ = df_sessions_for_good_.set_index('datetime_rnd')
# tasks_done_per_days_ = tasks_done_per_days_.set_index('datetime_rnd')



emotions_habits_t = emotions_habits.reset_index().copy()
emotions_habits_t.datetime_rnd = emotions_habits_t.datetime_rnd.astype(str)

tasks_done_per_days_.datetime_rnd = tasks_done_per_days_.datetime_rnd.astype(str)
# emotions_habits_t

df_sessions_for_good_t = df_sessions_for_good_.reset_index().copy()
df_sessions_for_good_t.datetime_rnd = df_sessions_for_good_t.datetime_rnd.astype(str)

#MERGING ALL together
df_final_ = emotions_habits_t.merge(tasks_done_per_days_, how='left', on='datetime_rnd')
df_final = df_final_.merge(df_sessions_for_good_t, how='left', on='datetime_rnd')


# df_final_ = pd.concat([emotions_habits, tasks_done_per_days_], axis = 1)
# df_final = pd.concat([df_final_, df_sessions_for_good_], axis = 1)





In [ ]:
df_sessions1

In [ ]:
emotions_habits.info()

In [ ]:
df_sessions_for_good[df_sessions_for_good.amount_sessions>0]

In [ ]:
emotions_habits

In [ ]:
tasks_done_per_days_


In [ ]:
# show distribution of hours when tasks were done in toggl data
df_sessions1.stop.dt.strftime("%H").astype(int).plot(kind="kde")



## POSSIBLE TO DRAW HOW SCHEDULE,  WORKHOURS  DEPENDS ON DONE TASKS

what work hours are more productive and how are they changing

In [ ]:
df_sessions1.stop.dt.strftime("%M").astype(int).plot(kind="kde")



In [ ]:
pd.to_datetime(str(random.randint(11, 22))+':'+str(random.randint(10, 43)) +':'+'00').strftime\
        ("%H:%M:%S")




In [ ]:
# FROM HERE WE ARE TAKING NUMBER OF SESSIONS IN SPECIFIC TIMESLOTS
#hours-minutes are not generated but summed by hours
df_sessions_for_good_ #.set_index('datetime_rnd', inplace=True)

In [ ]:
# MAIN DF WITH EMOTIONS HABITS PER TIMESLOTS
emotions_habits

In [ ]:
tasks_done_per_days_




In [ ]:

df_final#[(df_final.amount_sessions.notna()) & (df_final.tasks_done.notna())]#.index # &  & df_final['5-7'].notna())
# df_final.tasks_done.notna()
#  df_final.tasks_done.notna()]  #
# .amount_sessions.value_counts()


In [ ]:
def number_done_tasks_per_d():
    from dateutil.relativedelta import relativedelta

    number_done_tasks_per_day = number_tasks_per_day(df_tasks_3_t).iloc[:,:1].reset_index()

    left_range = number_done_tasks_per_day.completed_datetime.max() + relativedelta(months=-1)
    right_range = number_done_tasks_per_day.completed_datetime.max()

    fig = px.line(number_done_tasks_per_day, x="completed_datetime",y=[number_done_tasks_per_day.subject,
                                                                       number_done_tasks_per_day.subject.rolling(30).mean()],
                  color_discrete_sequence=px.colors.sequential.Plasma_r,
                  template="plotly_dark", line_shape='spline',
                  range_x=(left_range,right_range),title='NUMBER_DONE_TASKS_PER_DAY')

    fig.update_layout(
            autosize=False,
            height=500,
            width=600,showlegend=False,
                # plot_bgcolor="white",
                margin=dict(t=35,l=15,b=15,r=15),
                xaxis_title='NUMBER_DONE_TASKS_PER_DAY',
                yaxis_title=''
    )
    fig.show()

number_done_tasks_per_d()

In [ ]:
def working_time_stats():
    """plotting 3 stats on everyday working sessions
    number of sessions per day
    amount of hours tracked per day
    average for 30days for daily workhours
    scaling for last month
    """
    from dateutil.relativedelta import relativedelta

    df_sessions['for_count']=df_sessions['subject'].map(type) == str
    #number of sessions per day
    number_sessions_per_day = df_sessions.groupby(pd.Grouper(key='start', axis=0,
                                         freq='D')).agg({'for_count':sum}).reset_index()

    #amount of hours tracked per day
    amount_of_hours_tracked_per_day = df_sessions.groupby([pd.Grouper(key='start', axis=0,
                                         freq='D'),]).agg({'duration':sum})/ 3600
    # number_sessions_per_day.start = number_sessions_per_day.start.astype(str)

    out=pd.concat([number_sessions_per_day.set_index('start'),amount_of_hours_tracked_per_day], axis=1).reset_index()
    print(out)

    left_range = out.start.max() + relativedelta(months=-1)
    right_range = out.start.max()

    fig19 = px.line(out, x=out.start, y=[out.for_count, out.duration, out.duration.rolling(30).mean()],
                            line_shape = 'spline',
                            title='sessions_info',
                            color_discrete_sequence=px.colors.sequential.Plasma_r,
                            template="plotly_dark", range_x=(left_range,right_range)
                     )
    fig19.update_layout(
            autosize=False,
            height=600,
            width=800,#showlegend=False,
                margin=dict(t=35,l=15,b=15,r=15),
                xaxis_title='',
                yaxis_title=''
        )

    fig19.show()


working_time_stats()

In [ ]:

def  life_areas_balance_count(df_tasks_3_t, df_habits, period='W'):
    """count habits and tasks done by time period grouping by life_areas
        not counting summary values of habbits, just counting amount of acts """
    from shapely.geometry import MultiPoint
    habits_list_for_check = {
    'PHYSICAL': ['slept_GOOD', 'cold shower', '10 pushups everyday', 'big physical activity',
                              'over_eated', 'fastfood', 'cafe'],

    'FUN_RECREATION': ['traveled', 'drugs', 'jrk', 'porn', 'too much movies',
    'too much youtube', 'too much social media', 'too much news', 'any pain',
    '+youtubed', 'film', 'tvshow', 'cinema', 'gaming'],

    'INTELECTUAL': ['made smth for selfefficiency',
    'how many info i consumed vs generated out in the world', 'running on plans feeling', 'psycho practices',
    '5 minute journal', 'meditation', 'look inside 4 feelings on whole life', 'bucket list',
    'wish list', 'reading', 'studing', 'chess', 'ankicards', 'languages'],

    'LOVE ROMANCE SEX': ['sex','new sex partner', 'new sex practices'],

    'PARTNER': ['harmony_pleasurefull', 'critiqued_by_HER',
    'critiqued_by_ME', 'arguing', 'work thru complicated situations', 'common goals completion'],

    'SOCIAL FRIENDS': ['social offline', 'new people', 'old_friends'],

    'FINANCIAL': ['encreased income','financial reduce costs', 'investing']
    }

    #done tasks by life_areas
    df_done_tasks_by_lifearea = amount_tasks_done_life_areas(df_tasks_3_t, time_period=period).iloc[:,:1].reset_index()\
        .rename(columns={'completed_datetime':'datetime_rnd','subject':'act'})
    # df_done_tasks_by_lifearea


    df_habits['life_area'] = ''
    for k in df_habits.act.unique(): # adding column life_area to each acts responsively
        for i in habits_list_for_check.keys():
            if k in habits_list_for_check[i]:
                df_habits.loc[df_habits['act'] == k,'life_area'] = i
    df_habits = df_habits[df_habits.life_area!=''] #dropping rows which arent in dictionary

    #grouping by period of time and life area and counting similar acts
    df_habits1 = df_habits.groupby([pd.Grouper(key='datetime_rnd', axis=0,
                                     freq=period), df_habits.life_area]).agg({'act':'value_counts'})

    df_habits1.index.names = ["datetime_rnd", "life_area", "acts"] # renaming duplicate columns
    df_habits1 = df_habits1.reset_index().groupby(['datetime_rnd', 'life_area']).agg({'act':sum}).reset_index()
    # print(df_habits)

    out = pd.concat([df_done_tasks_by_lifearea, df_habits1])
    out = out.groupby(["datetime_rnd","life_area"], as_index=False).sum()


    # making mean values for year
    out_mean_year = out.groupby([pd.Grouper(key='datetime_rnd', axis=0,
                                     freq='Y'), out.life_area]).agg({'act':'mean'}).reset_index()
    # out_mean_year.rename(columns={'act': 'act_mean'}, inplace=True)
    out_mean_year.act_mean = out_mean_year.act.round(0).astype(int)
    out_mean_year.datetime_rnd = out.datetime_rnd.mean()
    out['model'] = 'daily'
    out_mean_year['model'] = 'year_mean'
    # print(pd.concat([out, out_mean_year], axis=0))
    # out = out.set_index('life_area').join(out_mean_year.set_index('life_area')).reset_index()

    out2=pd.DataFrame()
    temp = out_mean_year.copy() # df to copy on all unique dates to show year mean values
                                                # constantly
    # print(out.life_area.unique())
    for ik in out.datetime_rnd.unique():
        temp.datetime_rnd = ik
        out2 = pd.concat([out2,temp], axis=0)

    # for il in out.act.unique():



    out=pd.concat([out, out2], axis=0)

    #preparing df for plotting
    out['datetime_rnd'] = out['datetime_rnd'].astype(str)
    out.datetime_rnd= out.datetime_rnd.astype('datetime64').dt.strftime('%Y-%m-%d')

    out.drop(out[out.life_area.isin(['FUNRECREATION', 'INTELLECTUAL'])].index, inplace=True)

    out = out.sort_values(by=['act', 'life_area'], ascending=True)

    out['act']=out['act']+0.3
    max_r = out.act.max()

    # https://stackoverflow.com/questions/73624867/how-to-calculate-area-of-a-radar-chart-in-plotly-matplotlib
    # compare areas of two plots
    # convert theta to be in radians
    out["theta_n"] = pd.factorize(out["life_area"])[0]
    out["theta_radian"] = (out["theta_n"] / (out["theta_n"].max() + 1)) * 2 * np.pi
    # work out x,y co-ordinates
    out["x"] = np.cos(out["theta_radian"]) * out["act"]
    out["y"] = np.sin(out["theta_radian"]) * out["act"]
    df_a = out.groupby("model").apply( lambda d: MultiPoint(list(zip(d["x"], d["y"]))).convex_hull.area)
    # print(df_a)
    print(out)

    fig12 = px.line_polar(out[out.datetime_rnd>'2022-07-14'],
                        r='act',
                        theta='life_area', color="model",
                        line_close=True,  line_shape = 'spline',
                        animation_frame = 'datetime_rnd',
                        title='Emotions',
                        range_r=[0,max_r],
                        start_angle=270,
                        color_discrete_sequence=px.colors.sequential.Plasma_r,
                        template="plotly_dark"
                 )



    fig12.update_layout(
        autosize=False,
        height=500,
        width=600
    )



    fig12.show()
    # pyo.plot(fig)
    fig12.write_html('plot_life_areas_balance.html')


life_areas_balance_count(df_tasks_3_t, df_habits, period='W')



In [ ]:
def ideas_sparks(df_tasks_3_t):
    """#creativity - ideas sparks
    amount of new ideas-tasks written down on TODO app"""
    from dateutil.relativedelta import relativedelta


    df_new_tasks_pday = amount_new_tasks_per_day(df_tasks_3_t)
    left_range = df_new_tasks_pday.created_datetime.max() + relativedelta(months=-1)
    right_range = df_new_tasks_pday.created_datetime.max()
    fig16 = px.line(df_new_tasks_pday, x="created_datetime",y=[df_new_tasks_pday.subject,
                                                                           df_new_tasks_pday.subject.rolling(30).mean()],
                    # color=["red", "goldenrod", "#00D"],
                      color_discrete_sequence=px.colors.qualitative.G10, #px.colors.sequential.Plasma_r,
                      template="plotly_dark", line_shape='spline',
                      range_x=(left_range,right_range) , title='NUMBER_DONE_TASKS_PER_DAY'
                    )

    fig16.update_layout(
                autosize=False,
                height=400,
                width=500,
                showlegend=False,
                # plot_bgcolor="white",
                margin=dict(t=35,l=15,b=15,r=15),
                xaxis_title='',
                yaxis_title=''
        )
    fig16.update_traces(line=dict(width=1.5), opacity=.9,)
    # fig16.update_xaxes(visible=False)
    # fig16.update_yaxes(visible=False)#, fixedrange=True
    fig16.show()

ideas_sparks(df_tasks_3_t)


In [ ]:
#TODO plot and see trend, timewindows????
#in how many times amount of not underrated tasks (predicted TTC right)
# more than amount of predicted badly tasks

df_tasks_3_t[df_tasks_3_t.underrated==False].underrated.count()/df_tasks_3_t[df_tasks_3_t.underrated==True].underrated.count()



In [ ]:

def cold_start_ml(df_tasks_3_t):

    most_forgotten = []
    most_active = []
    # recommend tasks from most forgotten areas
    for iii in count_balance_of_life_areas_tasks_habits(df_tasks_3_t)[0]:
        if df_tasks_3_t[df_tasks_3_t['life_area'] == iii].shape[0] !=0:
            # print(iii)
            # print(' ------  ')
            # print(return_tasks_list_by(df_tasks_3_t, iii).subject.sample())
            most_forgotten.append('----TOP from ' + str(iii) + ':' )
            # print('TOP from ' + str(iii) + ':' )
            # print(return_tasks_list_by(df_tasks_3_t, iii).subject.head(3))
            if type((return_tasks_list_by(df_tasks_3_t, iii)))==tuple:
                for kk in return_tasks_list_by(df_tasks_3_t, iii):
                    # print(k)
                    most_forgotten.extend(kk.to_list())
                    # most_forgotten += "\n"
                # print(most_forgotten)

            else:
                for ki in return_tasks_list_by(df_tasks_3_t, iii).subject.values:
                    # print(k)
                    most_forgotten.append(ki)
                    # most_forgotten += "\n"
                # print(most_forgotten)


        # recommend tasks from most active areas
    for iii in count_balance_of_life_areas_tasks_habits(df_tasks_3_t)[1]:
        if df_tasks_3_t[df_tasks_3_t['life_area'] == iii].shape[0] !=0:

            # print(iii)
            # print(' ------  ')
            most_active.append('-----TOP from ' + str(iii) + ':')
            # most_active += "\n".join(return_tasks_list_by(df_tasks_3_t, iii))
            # most_active += "\n"
            # print(most_active)
            if len(return_tasks_list_by(df_tasks_3_t, iii))!=0:
                if type((return_tasks_list_by(df_tasks_3_t, iii)))==tuple:
                    for k in return_tasks_list_by(df_tasks_3_t, iii):
                        most_active.extend(k.to_list())
                        # print(most_active)
                        # print(k)
                # else:
                #     for ke in return_tasks_list_by(df_tasks_3_t, iii).subject.values:
                #         # print(k)
                #         most_forgotten.append(ke)

    return most_forgotten,most_active

cold_start_ml(df_tasks_3_t)[1]

In [ ]:
cold_start_ml(df_tasks_3_t)[0]


In [ ]:
df_sessions[df_sessions.duration<0]

In [ ]:
import time
from gcal import *
from gcsa.reminders import EmailReminder, PopupReminder



def add_popup_reminder(name: str, calendar_name, start_time):
    """
    adding reminders to calendar

    #TODO define list of forgotten habits
    #TODO define time to remind - small timeslots(?)

    :param name: title of reminder
    :param calendar_name: calendar type name
    :param start_time: time of reminder
    :return:
    """

    popup = Event(name,
                  start=start_time,
                  reminders=[
                        PopupReminder(minutes_before_start=1)
                  ])
    calendar_name.add_event(popup)
    print('POPUP added' + " " + name)

# add_popup_reminder('blablalba', main_calendar, D.now()+2*minutes)

def delete_calendar_tasks(calendar):
    """delete all events in calendar"""

    for ev in calendar:
        calendar.delete_event(ev)


def get_list_tasks_today(calendar):
    """without repetitions"""

    ll = set([e.summary for e in calendar[D.now():D.now()+ 1* days:'updated']])
    return ll



def put_tasks1(list_of_tasks_names, df_tasks11, calendar_name):
    """
    Taking list of predicted task names and pulling them to the Gcalendar
    checking for 1 day window - if there are tasks?
    if there are - getting the free intervals checking their size
    if no enough sized timeslots, putting tasks at the end, after last task

    :param list_of_tasks_names: list of predicted task names
    :param df_tasks11: dataframe to get TTC metric for putting into calendar
    :return:
    """

    def how_many_tasks():
        return len(find_timeslot_for_task(calendar_name)[1])

    for i in set(list_of_tasks_names[:5]): # take first 5 task of sorted tasks
        if '----' not in i:
            num_tasks = how_many_tasks()
            gaps = find_timeslot_for_task(calendar_name)[0]

            if num_tasks == 0: # if schedule in 3 day range ( see find_timeslot_for_task() ) is empty
                if D.today()[11] > D.now():
                    start_task_time = D.today()[11]
                else:
                    start_task_time = D.now() + 10 * minutes

                if df_tasks11[df_tasks11.subject == i].TTC.notna().values.any():   # check if there is TTC metric
                    task_duration = int(df_tasks11.loc[df_tasks11.subject==i].TTC.values[0])
                else:
                    task_duration = 1 # default time period for small not TTC defined task

                if task_duration <= 1:  # if work time of task dont need to be sliced
                    add_sessions_to_gcal(calendar_name, name=i, start_time=start_task_time,
                                         end_time=start_task_time + task_duration * 60 * minutes)
                    time.sleep(5)
                    continue

                elif task_duration > 1:     # if task need to be divided
                    number_of_sprints = task_duration.round(0) + 1
                    for kk in range(number_of_sprints): # adding task divided in 1-hour sprints
                        add_sessions_to_gcal(calendar_name, name=i, start_time=start_task_time,
                                             end_time=start_task_time + 70 * kk * minutes)
                        time.sleep(5)
                        continue

            elif num_tasks == 1:  # if schedule is not empty
                add_sessions_to_gcal(calendar_name, name=i,
                                     start_time=find_timeslot_for_task(calendar_name)[2][0] + 10 * minutes,
                                     end_time=find_timeslot_for_task(calendar_name)[2][0] + 70 * minutes)
                time.sleep(5)
                continue

            elif num_tasks > 1:     # if schedule is not empty
                if df_tasks11[df_tasks11.subject == i].TTC.notna().values.any():
                    task_duration = int(df_tasks11.loc[df_tasks11.subject==i].TTC.values[0])
                else:
                    task_duration = 1

                i_ = 0  # number of times one task added
                n_gap = 0  # index of gap

                for gap in gaps:   # search for the right gap for task
                    if gap.seconds/3600 >= task_duration and i_ < 1:
                        print(gap.seconds//3600)
                        print(task_duration)

                        add_sessions_to_gcal(calendar_name,
                                        name=i,
                                        start_time=find_timeslot_for_task(calendar_name)[2][n_gap] + 10 * minutes,
                                        end_time=find_timeslot_for_task(calendar_name)[2][n_gap] + 70 * minutes)
                        time.sleep(2)
                        i_ += 1
                        n_gap += 1
                        continue

                    else: # if no right gap add to the end
                        print('too small gap OR already added')
                        n_gap += 1
                        continue

                if i_ < 1:  # in case task never added to the gaps add it to the end
                    print('adding to the end')

                    add_sessions_to_gcal(calendar_name,
                                         name=i,
                                         start_time=find_timeslot_for_task(calendar_name)[2][-1] + 10 * minutes,
                                         end_time=find_timeslot_for_task(calendar_name)[2][-1] + 70 * minutes)




In [ ]:

most_forgotten, most_active = cold_start_ml(df_tasks_3_t)

list_of_tasks = cold_start_ml(df_tasks_3_t)[1]
sessions_calendar = GoogleCalendar(config.get('GCALENDAR', 'sessions'), credentials_path='credentials.json')

delete_calendar_tasks(sessions_calendar)
time.sleep(10)


# put_tasks1(list_of_tasks_names=most_forgotten, df_tasks11=df_tasks_3_t, calendar_name=sessions_calendar)
# time.sleep(20)

# put_tasks1(list_of_tasks_names=most_active, df_tasks11=df_tasks_3_t, calendar_name=sessions_calendar)
# put_tasks1(list_of_tasks_names=list_of_tasks, df_tasks11=df_tasks_3_t, calendar_name=sessions_calendar)



In [ ]:
df_tasks_3_t[df_tasks_3_t.subject=='MY CV REVIEW'].TTC.notna().values.any()

In [ ]:
# import plotly.graph_objects as go

categories = ['Bitterness','Froth','Price',
              'Content', 'Mouthfeel']



fig12 = px.line_polar(out,
                        r=out.act,
                        theta=out.life_area,
                        line_close=True,  line_shape = 'spline',
                        animation_frame = out.datetime_rnd,
                        title='Emotions',
                        # fill='toself',
                        # fillcolor='rgba(0, 255, 0, 0.4)',
                        # range_r=(0,2),
                        color_discrete_sequence=px.colors.sequential.Plasma_r,
                        template="plotly_dark",
                 )


fig12 = go.Figure()

fig12.add_trace(go.Scatterpolar(
                        r=out.act,
                        theta=out.life_area,
                        line_close=dict({True:1}),  line_shape = dict({'spline':1}),
                        animation_frame = out.datetime_rnd,
                        # title='Emotions',
                        # fill='toself',
                        # fillcolor='rgba(0, 255, 0, 0.4)',
                        # range_r=(0,2),
                        color_discrete_sequence=px.colors.sequential.Plasma_r,
                        template=dict({"plotly_dark":1}),
))
fig12.add_trace(go.Scatterpolar(
      r=[4, 3, 2.5, 1, 2],
      theta=categories,
      fill='toself',
      name='Product B'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 5]
    )),
  showlegend=False
)

fig.show()

In [ ]:
df

In [ ]:
#only from sessions df
number_tasks_per_day_per_task(df_tasks_3_t).reset_index()



In [ ]:
number_hard_tasks_per_m(df_tasks_3_t).iloc[:,:1].reset_index()

In [ ]:
#TODO WTFFFFF
check_goals_difficulty(df_tasks_3_t)

In [ ]:
tasks_done_per_month(df_tasks_3_t).iloc[:,:1]

In [ ]:
amount_complited_tasks_permonth(df_tasks_3_t).iloc[:,:1].reset_index()

In [ ]:
amount_complited_tasks_per_list_month(df_tasks_3_t, 'BACKLOG life').iloc[:,:1].reset_index()

In [ ]:
#creativity as addded tasks to todo lists
creat = amount_new_tasks_per_day(df_tasks_3_t)
mean_creat = creat.subject.mean()
today_creat = creat.subject.iloc[-1]
print(mean_creat)
print(today_creat)

In [ ]:
df_tasks_3_t[df_tasks_3_t.underrated==False]


In [ ]:
df_tasks_3_t[df_tasks_3_t.metricks.notnull()]


In [ ]:
df_tasks_3_t[(df_tasks_3_t.underrated==False)&(df_tasks_3_t.status=='NotStarted')]


In [ ]:
df_sessions.groupby(['subject']).agg({'duration':sum})#.reset_index().mean()/3600

In [ ]:
df_sessions

In [ ]:
#TODO Make timeseries for emotion balance.
# Associate ((????)) connect it to sessions time



In [ ]:
#trying to visualise
#try to astype linux datetime format for x for go.Violin

import plotly.graph_objects as go
from plotly.colors import n_colors
from dateutil.relativedelta import relativedelta

fig = go.Figure()
colors = n_colors('rgb(5, 200, 200)', 'rgb(200, 10, 10)', 30, colortype='rgb')

left_range = df_emotions1["datetime_rnd"].max() + relativedelta(months=-1)
right_range = df_emotions1["datetime_rnd"].max()

for i,color in zip(df_emotions1.iloc[:,2:], colors):
    # y=df_emotions1.loc[:,i],
    # fig.add_trace(go.Violin(x=df_emotions1["datetime_rnd"][109:].astype(str), y=np.array(df_emotions1.loc[109:,i]),
    #                         line_color=color))#, name=i

    fig.add_trace(go.Scatter(x=df_emotions1["datetime_rnd"], y=df_emotions1.loc[:,i],
                             name=i, mode="lines",
                             )) #, line_color=color line_shape ='spline'

fig.update_layout(
        title="Emotions", xaxis_title="Date", yaxis_title="Close",
        template="plotly_dark",yaxis_range=[0,1.5], xaxis_range=[left_range,right_range]
    )
# fig.update_traces(orientation='h', side='positive', width=2, points=False)
# fig.update_layout(xaxis_showgrid=False, xaxis_zeroline=False)
fig.show()

In [ ]:
data

In [ ]:
tttest+1

In [ ]:
tttest = df_emotions1.copy().drop(columns = {'datetime_rnd',
                   'act'}).iloc[5:37,:].T
tttest = np.array(tttest+1)


import plotly.graph_objects as go
from plotly.colors import n_colors
import numpy as np
np.random.seed(1)

# 12 sets of normal distributed random data, with increasing mean and standard deviation
data = (np.linspace(1, 2, 12)[:, np.newaxis] * np.random.randn(12, 200) +
            (np.arange(12) + 2 * np.random.random(12))[:, np.newaxis])
colors = n_colors('rgb(5, 200, 200)', 'rgb(200, 10, 10)', 32, colortype='rgb')
# print(data[:3])

fig = go.Figure()
for data_line, color in zip(tttest, colors):
    print(data_line)
    print(color)
    fig.add_trace(go.Violin(x=data_line, line_color=color))

fig.update_traces(orientation='h', side='positive', width=3, points=False)
fig.update_layout(xaxis_showgrid=False, xaxis_zeroline=False,template="plotly_dark")
fig.show()



In [ ]:
import plotly.graph_objects as go
from plotly.colors import n_colors
import numpy as np
np.random.seed(1)

# 12 sets of normal distributed random data, with increasing mean and standard deviation
data = (np.linspace(1, 2, 12)[:, np.newaxis] * np.random.randn(12, 200) +
            (np.arange(12) + 2 * np.random.random(12))[:, np.newaxis])
colors = n_colors('rgb(5, 200, 200)', 'rgb(200, 10, 10)', 12, colortype='rgb')
# print(data[:3])

fig = go.Figure()
for data_line, color in zip(data, colors):
    # print(data_line)
    # print(color)
    fig.add_trace(go.Violin(x=data_line, line_color=color))

fig.update_traces(orientation='h', side='positive', width=3, points=False)
fig.update_layout(xaxis_showgrid=False, xaxis_zeroline=False,template="plotly_dark")
fig.show()

In [ ]:
pd.DataFrame(data)


In [ ]:
pd.DataFrame(np.linspace(1, 2, 12)[:, np.newaxis] * np.random.randn(12, 200) +
            (np.arange(12) + 2 * np.random.random(12))[:, np.newaxis])


In [ ]:
# inspiration https://medium.com/@marcosanchezayala/plotting-pokemon-attributes-plotly-polar-plots-and-animations-319934b60f0e
#https://stackoverflow.com/questions/71781424/i-want-to-make-an-animated-polar-chart-but-the-chart-i-get-only-has-one-radii-wi



# preparing DFs
df_emotions2 = df_emotions1.drop(columns=['act'])
# df_emotions2
df_emotions2 = pd.melt(df_emotions2, id_vars='datetime_rnd', var_name='attribute', value_name='attribute_value')
df_emotions2['datetime_rnd'] = df_emotions2['datetime_rnd'].astype(str)
df_emotions2['attribute_value']=df_emotions2['attribute_value']+0.5

import plotly.express as px
# import plotly.offline as pyo

fig = px.line_polar(df_emotions2,
                    r='attribute_value',
                    theta='attribute',
                    line_close=True,  line_shape = 'spline',
                    animation_frame = 'datetime_rnd',
                    title='Emotions',
                    range_r=(0,2),
                    color_discrete_sequence=px.colors.sequential.Plasma_r,
                    template="plotly_dark",
             )
fig.update_layout(
    autosize=False,
    height=600,
    width=800,
)

fig.show()
# pyo.plot(fig)
fig.write_html('plot_emotions.html')

In [ ]:
#PLOTTING ANIMATED TIMESERIES FOR habitts ARCHIVE

# inspiration https://medium.com/@marcosanchezayala/plotting-pokemon-attributes-plotly-polar-plots-and-animations-319934b60f0e
#https://stackoverflow.com/questions/71781424/i-want-to-make-an-animated-polar-chart-but-the-chart-i-get-only-has-one-radii-wi



#one-hot encoding for trying get data for plotting timeseries on radial plot
#getting dummies from list from grouped by rounded 10m datetime periods
#https://stackoverflow.com/questions/29034928/pandas-convert-a-column-of-list-to-dummies

df_habits1 = df_habits.groupby('datetime_rnd')['act'].agg(list).reset_index()#.iloc[1,:]

# GENERATION getdummies from list of emotions (act column) grouped by 10min periods
df_habits1 = df_habits1.join(df_habits1['act'].str.join('|').str.get_dummies())
# df_habits1




# preparing DFs
df_habits2 = df_habits1.drop(columns=['act'])
# df_emotions2
df_habits2 = pd.melt(df_habits2, id_vars='datetime_rnd', var_name='attribute', value_name='attribute_value')
df_habits2['datetime_rnd'] = df_habits2['datetime_rnd'].astype(str)
#adding 1 just for looking better
df_habits2['attribute_value']=df_habits2['attribute_value']+0.5

import plotly.express as px
# import plotly.offline as pyo

fig = px.line_polar(df_habits2,
                    r='attribute_value',
                    theta='attribute',
                    line_close=True,  line_shape = 'spline',
                    animation_frame = 'datetime_rnd',
                    title='Habits',
                    range_r=(0,2),
                    color_discrete_sequence=px.colors.sequential.Plasma_r,
                    template="plotly_dark",
             )
fig.update_layout(
    autosize=False,
    height=600,
    width=800,
)

fig.show()
# pyo.plot(fig)
fig.write_html('plot_habitts.html')

In [ ]:
#feature hashing instead of onehot encoding
#only for ML stage, cous of not interprateble
# https://www.kaggle.com/code/dansbecker/using-categorical-data-with-one-hot-encoding/notebook
# https://scikit-learn.org/stable/modules/feature_extraction.html#feature-hashing



from sklearn.feature_extraction import FeatureHasher
# n_features contains the number of bits you want in your hash value.
h = FeatureHasher(n_features = 15, input_type ='string')

# transforming the column after fitting
hashed_Feature = h.fit_transform(df_emotions['act'])

hashed_Feature = hashed_Feature.toarray()
pd.DataFrame(hashed_Feature)
# df___ = pd.concat([df_emotions, pd.DataFrame(hashed_Feature)], axis = 1)
# df___.head(10)



In [ ]:
#https://towardsdatascience.com/4-categorical-encoding-concepts-to-know-for-data-scientists-e144851c6383
#
# import category_encoders as ce
# encoder=ce.HashingEncoder(cols='model_year',n_components=5)
# hash_res = encoder.fit_transform(mpg['model_year'])
# hash_res.sample(5)


In [ ]:
#auto feature engeneering
#https://towardsdatascience.com/easy-automated-feature-engineering-for-machine-learning-model-ea00c5059dd6


In [ ]:
# df_sessions.groupby(['description']).head(111)
#


In [ ]:
# df_sessions['description'].iloc[1:]
#

In [ ]:
# df_sessions_tasks_duration.reset_index()

In [ ]:
# list_of_cumsum_sessions = df_sessions_tasks_duration.reset_index().subject.tolist()
#
# df_tasks_3['duration'] = df_sessions_tasks_duration.reset_index() for session_name in list_of_cumsum_sessions
# df_tasks_3.join(df_sessions_tasks_duration, on='subject', how='left')

In [ ]:
# pd.merge(df_tasks_3 , df_sessions_tasks_duration.reset_index() , on=['subject'])
#
#
#

In [ ]:
# list_of_cumsum_sessions = df_sessions_tasks_duration.reset_index().subject.tolist()
# # df_tasks_3

In [ ]:
# df_sessions_tasks_duration.reset_index()['subject']=='ORCHESTRATION 10 3 4 3 3'
#

In [ ]:
# pd.merge(df_tasks_3, df_sessions_tasks_duration.reset_index(),left_on='subject',right_on='subject', how='left')

In [ ]:


# df_sessions_tasks_duration = df_sessions.groupby(['subject'])['duration'].sum()
#
# test = pd.merge(df_tasks_3, df_sessions_tasks_duration.reset_index(),left_on='subject',right_on='subject', how='left')



In [ ]:
# test

In [ ]:
# test['underrated'] = (test['duration'].astype('Int32') / 3600 - test['TTC'].astype('Int32')) > (0.15 *
#                                                                                                  test['TTC'].astype(
#                                                                                                      'Int32')+0.01)


In [ ]:
# test[test['underrated']==True]


In [ ]:
# test[test['metricks'].astype('Int32')!=0]


In [ ]:
# test.info()


In [ ]:
# df_tasks_3
# TTC	PRI	DIFF PLEAS RESIS

In [ ]:
#one-hot encoding for trying get data for plotting timeseries on radial plot

# from sklearn.preprocessing import OneHotEncoder
# enc = OneHotEncoder()
# # transforming the column after fitting
# enc = enc.fit_transform(df_emotions[['act']]).toarray()
# # converting arrays to a dataframe
# encoded_colm = pd.DataFrame(enc)
# # concatenating dataframes
#
# # encoded_colm faster
# df_emotions1 = pd.concat([df_emotions, encoded_colm], axis=1)
# df_emotions1
# # # removing the encoded column.
# # df = df.drop(['nom_0'], axis=1)
# # df.head(10)


In [ ]:
import plotly.express as px
df = px.data.wind()
fig = px.line_polar(df, r="frequency", theta="direction", color="strength", line_close=True,
                    color_discrete_sequence=px.colors.sequential.Plasma_r,
                    template="plotly_dark",)
fig.show()


In [ ]:
df.head(10)

In [ ]:
import plotly.express as px

df = px.data.gapminder()
fig = px.scatter(df, x="gdpPercap", y="lifeExp", animation_frame="year", animation_group="country",
           size="pop", color="continent", hover_name="country",
           log_x=True, size_max=55, range_x=[100,100000], range_y=[25,90])

# fig["layout"].pop("updatemenus") # optional, drop animation buttons
fig.show()


In [ ]:
df.head()


In [ ]:
import plotly.graph_objects as go
import plotly.offline as pyo


categories = ['Food Quality', 'Food Variety', 'Service Quality', 'Ambience', 'Affordability']
categories = [*categories, categories[0]]

restaurant_1 = [4, 4, 5, 4, 3]
restaurant_2 = [5, 5, 4, 5, 2]
restaurant_3 = [3, 4, 5, 3, 5]
restaurant_1 = [*restaurant_1, restaurant_1[0]]
restaurant_2 = [*restaurant_2, restaurant_2[0]]
restaurant_3 = [*restaurant_3, restaurant_3[0]]


fig = go.Figure(
    data=[
        go.Scatterpolar(r=restaurant_1, theta=categories, name='Restaurant 1'),
        go.Scatterpolar(r=restaurant_2, theta=categories, name='Restaurant 2'),
        go.Scatterpolar(r=restaurant_3, theta=categories, name='Restaurant 3')
    ],
    layout=go.Layout(
        title=go.layout.Title(text='Restaurant comparison'),
        polar={'radialaxis': {'visible': True}},
        showlegend=True
    )
)

pyo.plot(fig)


In [ ]:
restaurant_1


In [ ]:
# df_tasks_3['PLEASURE_aft'] = None
# for i in df_tasks_3[df_tasks_3.status=='Completed'].subject:


# count_emotion_balance_of_task_completion_day_by_task_name('REFACTORING 4 3 2 2 3')
#
#
# dateee=date_of_task_completion('guberman goal setting video').iloc[0]
#
# df_emotions1[df_emotions1['datetime_rnd'].dt.strftime("%Y-%m-%d")==dateee].act.sum()
#
# indd = df_tasks_3[df_tasks_3.subject=='REFACTORING 4 3 2 2 3'].index
#
# df_tasks_3.iloc[indd,:].PLEASURE_aft

# date_of_task_completion('test forked running bot')#.astype('datetime64')
# df_emotions1[df_emotions1['datetime_rnd'].iloc[1].strftime("%Y-%m-%d")==date_of_task_completion('test forked running bot').iloc[0]]
#
# df_tasks_3[df_tasks_3.status=='Completed']
# df_emotions1.datetime_rnd.iloc[1].strftime("%Y-%m-%d")
# date_of_task_completion('test forked running bot').iloc[0]
# df_emotions1['datetime_rnd'].astype(str)#.iloc[1].strftime("%Y-%m-%d")

In [ ]:
# def first_session_date_by_taskname(taskname):
#     """return date of first session of task"""
#     if len(df_sessions[df_sessions.subject==taskname].start.agg(list))==0:
#         return None
#     else:
#         return min(df_sessions[df_sessions.subject==taskname].start.agg(list))
#
# # first_session_date_by_taskname('calendar sync visualisation 4 2 3 4 1')
# # df_sessions[df_sessions.subject=='calendar sync visualisation 4 2 3 4 1'].head()
#
# # load_emotions_habits_values()
#
# def date_of_task_completion(taskname):
#     """take date of completed task"""
#     mask__ = (df_tasks_3.subject==taskname)
#     #&(isinstance(df_tasks_3[df_tasks_3.subject==taskname].completed_datetime,datetime.datetime))
#     return df_tasks_3[mask__].completed_datetime
#
# # date_of_task_completion('test forked running bot').astype('datetime64') #check
#
#
# def common_list_items(a,b):
#     from collections import Counter
#     ca = Counter(a)
#     cb = Counter(b)
#
#     result = [a for b in ([key] * min(ca[key], cb[key])
#                           for key in ca
#                           if key in cb) for a in b]
#     return result
#
#
# #TODO create mean of emotion balance during ALL task sessions
# def count_emotion_balance_of_task_completion_day_by_task_name(taskname, regime=0, values_list=[]):
#     """get list of emotions during !!!! COMPLETION_day!!!!! by taskname"""
#
#     global emotions_values, habits_values, emotions_and_habits_values
#
#
#
#     dateee=date_of_task_completion(taskname).iloc[0]
#     # print(dateee)
#     # print(df_emotions1['datetime_rnd'].dt.strftime("%Y-%m-%d"))
#     mask___ = (df_emotions1['datetime_rnd'].dt.strftime("%Y-%m-%d")==dateee)
#     # print(mask___)
#     daily_list_of_emotions = df_emotions1[mask___].act.sum()
#     # print(dayly_list_of_emotions)
#     daily_emotional_balance = 0
#
#     #not all tasks are completed after i started tracking emotions,
#     # so check if 0 registered emotions
#     if daily_list_of_emotions!=0 and regime==0:
#         for ik in emotions_values.keys():
#             if ik in daily_list_of_emotions:
#                 daily_emotional_balance += emotions_values[ik]
#             return daily_emotional_balance
#     #part for counting frustration and procrastination
#     else:
#         if daily_list_of_emotions!=0 and regime==1:
#             for ik in common_list_items(emotions_and_habits_values.keys(),values_list):
#                 if ik in daily_list_of_emotions:
#                     daily_emotional_balance += emotions_and_habits_values[ik]
#             return daily_emotional_balance
#     # print(daily_emotional_balance)
#



In [ ]:
# #ADDING MORE FEATURES for assuming metrics of tasks, based on secondary features
#
# # TTC_aft - sessions datetime sum if task done
# df_tasks_3_t = after_metric_ttc(df=df_sessions, df1=df_tasks_3)
# # df_tasks_3_t[df_tasks_3_t.metricks.notnull()] #checking
#
# df_tasks_3_t['completed_datetime'] = df_tasks_3_t['completed_datetime'].astype('datetime64')
# df_tasks_3_t['created_datetime'] = df_tasks_3_t['created_datetime'].astype('datetime64')
#
# # PRI_aft - compared date started(first session in toggl)-created /time done for each task compare
#
# #TODO check ! why there is so small amount of counted cells
# for i in df_tasks_3_t[df_tasks_3_t.status=='Completed'].subject:
#     if isinstance(first_session_date_by_taskname(i),datetime.datetime):
#         mask_ = (df_tasks_3_t.subject==i)
#         df_tasks_3_t['PRI_aft'] = (df_tasks_3_t[mask_].completed_datetime -
#                                                      first_session_date_by_taskname(i).replace(tzinfo=None) )/(df_tasks_3_t[mask_].created_datetime - first_session_date_by_taskname(i).replace(tzinfo=None))
#
#
#
# # PLEASURE_aft - end day emotional balance
# #TODO, or mean of all days when were sessions of that task)
# # FOR that ONE  is NEEDED df_emotions1 and df_tasks_3 DFs
# # end day emotional balance (TODO, or mean of all days when were sessions of that task)

#
# #emotion balance by date of completed task
# for k in df_tasks_3_t[df_tasks_3_t.status=='Completed'].subject:
#     # print(k)
#     indd = df_tasks_3_t[df_tasks_3_t.subject==k].index
#     df_tasks_3_t.loc[indd, 'PLEASURE_aft'] = count_emotion_balance_of_task_completion_day_by_task_name(k, None)
#
#
# # df_tasks_3_t[df_tasks_3_t.status=='Completed'] #cheking
#
#
# # DIFCLT_aft -  select frustration type of emotions, return were they there,
# #                   input list of sessions dates of certain
# #                   list of emotions pleasuring or stress  for certain task done
# # RES_aft amount of procrastination, too much youtube, or mobile screentime(????)
#
#
#
# # make lists for frustration and procrastination
# frustration = ['burnouted', 'emotionally UNbalanced', 'anxiety', 'confusion', '-fear', 'sadness', '-surprise',
#                    'dissatisfaction']
# procrastination = ['burnouted', 'procrastinated', 'boredom', 'horror', 'jrk', 'porn', 'too much movies',
#                        'too much youtube', 'too much social media', 'too much news']

# #TODO, when data on completed tasks will be more, what happening with that columns
# #count daily amount of such emotions
# for k in df_tasks_3_t[df_tasks_3_t.status=='Completed'].subject:
#     # print(k)
#     indd = df_tasks_3_t[df_tasks_3_t.subject==k].index
#     df_tasks_3_t.loc[indd, 'DIFCLT_aft'] = count_emotion_balance_of_task_completion_day_by_task_name(k, regime=1,
#                                                                                                      values_list=frustration)
#     df_tasks_3_t.loc[indd, 'RES_aft'] = count_emotion_balance_of_task_completion_day_by_task_name(k, regime=1,
#                                                                                                      values_list= procrastination)
#
# # df_tasks_3_t[df_tasks_3_t.DIFCLT_aft.notna()] #check
#
#
#
# #underrated as for TTC metric
# check_error_rate_predicting_ttc(df1=df_tasks_3_t)



In [ ]:
#TESTING PD.MELT fUNCTION
# "D:\downloads\artists.csv"
# "D:\downloads\tracks.csv"
# artists = pd.read_csv(r"D:\downloads\artists.csv")
spotify_data = pd.read_csv(r"D:\downloads\tracks.csv").head(10000)
spotify_data.shape

In [ ]:
med_spotify = spotify_data.drop(columns=['name', 'explicit', 'artists', 'key', 'mode', 'time_signature', 'tempo', 'duration_ms', 'loudness'])


med_spotify

In [ ]:
med_spotify = med_spotify.groupby('release_date').median()
# med_spotify.reset_index(inplace=True)
med_spotify

In [ ]:
med_spotify.reset_index(inplace=True)
med_spotify

In [ ]:
tetete = pd.melt(med_spotify, id_vars='release_date', var_name='attribute', value_name='attribute_value')
tetete.info()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

med_spotify = spotify_data.drop(columns=['name', 'explicit', 'artists', 'key', 'mode', 'time_signature', 'tempo', 'duration_ms', 'loudness'])
med_spotify = med_spotify.groupby('release_date').median()
med_spotify.reset_index(inplace=True)
plot_data = pd.melt(med_spotify, id_vars='release_date', var_name='attribute', value_name='attribute_value')

import plotly.express as px

fig = px.line_polar(plot_data,
                    r='attribute_value',
                    theta='attribute',
                    line_close=True,
                    animation_frame = 'release_date',
                    title='Spotify',
                    template = 'ggplot2'
             )
fig.update_layout(
    autosize=False,
    height=600,
    width=800,
)

fig.show()

In [ ]:
# plotting of 1strow - period in df
#take only first row and make it 'vertical'
test_graph = df_emotions1.iloc[0,:].reset_index().iloc[2:,:]
# test_graph

# inspiration https://medium.com/@marcosanchezayala/plotting-pokemon-attributes-plotly-polar-plots-and-animations-319934b60f0e
#https://stackoverflow.com/questions/71781424/i-want-to-make-an-animated-polar-chart-but-the-chart-i-get-only-has-one-radii-wi


import plotly.express as px

fig = px.line_polar(test_graph, r=0, theta="index",# color="strength", animation_frame="year",
                    line_shape = 'spline', line_close=True,range_r=(0,2),
                    color_discrete_sequence=px.colors.sequential.Plasma_r,
                    template="plotly_dark",)
fig.show()


In [ ]:

# from prefect import task, flow
# from UI_orchestration import *


# import streamlit.cli as cli
#
# import sys
#
#
# sys.argv = ['0','run','myApp']
# name = "main"
#
# cli.main()


# import sys
# from streamlit import cli as stcli
#
# if __name__ == '__main__':
#     sys.argv = ["streamlit", "run", "APP_NAME.py"]
#     sys.exit(stcli.main())

#
# import sys
#
# # total arguments
# n = len(sys.argv)
# print("Total arguments passed:", n)
#
# # Arguments passed
# print("\nName of Python script:", sys.argv[0])
#
# print("\nArguments passed:", end=" ")
# for i in range(1, n):
#     print(sys.argv[i], end=" ")
#
# # Addition of numbers
# Sum = 0
# # Using argparse module
# for i in range(1, n):
#     Sum += int(sys.argv[i])
#
# print("\n\nResult:", Sum)
#
# # Python program to demonstrate
# # command line arguments

#
# import getopt, sys
#
# # Remove 1st argument from the
# # list of command line arguments
# argumentList = sys.argv[1:]
#
# # Options
# options = "cl:"
#
# # Long options
# long_options = ["cloud", "local", "Output="]
#
# try:
#     # Parsing argument
#     arguments, values = getopt.getopt(argumentList, options, long_options)
#
#     # checking each argument
#     for currentArgument, currentValue in arguments:
#
#         if currentArgument in ("-l", "--local"):
#             print("Running Streamlit dashboard locally"
#                   "visit   -    localhost:")
#
#         elif currentArgument in ("-c", "--cloud"):
#             print("Running backend in cloud")
#             print("")
#
#         elif currentArgument in ("-h", "--help"):
#             print(("for help and documentation visit "
#                    "link"
#                    "link"
#                    "link")
#
# except getopt.error as err:
#     # output error, and return with an error code
#     print(str(err))